In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [2]:
import networkx as nx
import csv

def build_multidigraph_from_csv(csv_file):
    G = nx.MultiDiGraph()

    with open(csv_file, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            # Add nodes (if not already added)
            G.add_node(row['starter_ID'], name=row['starter_ID'])
            G.add_node(row['receiver_ID'], name=row['receiver_ID'])

            # Add directed edges with additional attributes
            # Each edge is unique and can represent a different type of interaction
            G.add_edge(
                row['starter_ID'], 
                row['receiver_ID'], 
                interaction_type=row['subtype_name'],
                relation_type=row['relation_type'],
                pathway_sources=row['pathway_source'],
                credibility=row['credibility']
            )

    return G

# Path to the CSV file
csv_file_path = 'relations_train_final.csv'  # Update this to the path of your relations_train.csv file

# Build the multidigraph
MDG = build_multidigraph_from_csv(csv_file_path)

# Print basic information about the multidigraph
print(f"Number of nodes: {MDG.number_of_nodes()}")
print(f"Number of edges: {MDG.number_of_edges()}")

# The multidigraph `MDG` is now constructed with nodes and multiple types of directed edges from the relations_train.csv
# You can now use `MDG` for further analysis or as input to clustering algorithms.



Number of nodes: 4816
Number of edges: 101782


In [3]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx

class GCN(torch.nn.Module):
    def __init__(self, num_nodes, embedding_dim, num_classes, num_hidden_units=32):
        super(GCN, self).__init__()
        self.node_emb = torch.nn.Embedding(num_nodes, embedding_dim)  # Trainable embeddings
        self.conv1 = GCNConv(embedding_dim, num_hidden_units)  # Adjusted to use num_hidden_units
        self.conv2 = GCNConv(num_hidden_units, num_classes)  # Input from the hidden layer

    def forward(self, data):
        x = self.node_emb(data.node_index)  # Node features
        edge_index = data.edge_index  # Edge indices

        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

# Assuming MDG is your MultiDiGraph
train_data = from_networkx(MDG)

num_nodes = train_data.num_nodes
embedding_dim = 32  # Choose an appropriate embedding size
num_classes = 7   # Set the number of classes based on your edge types


In [4]:
# Collect all unique interaction types from MDG
interaction_types = set()
for _, _, edge_data in MDG.edges(data=True):
    interaction_types.add(edge_data['interaction_type'])

# Update the mapping to include 'no interaction' class
interaction_type_to_label = {inter_type: i for i, inter_type in enumerate(interaction_types)}
num_classes = len(interaction_type_to_label)

def setup_edge_labels_with_no_interaction(MDG, interaction_type_to_label, data):
    num_nodes = len(MDG.nodes())
    edge_labels = torch.zeros((num_nodes, num_nodes), dtype=torch.long)

    for u, v, edge_data in MDG.edges(data=True):
        u_index = list(MDG.nodes()).index(u)
        v_index = list(MDG.nodes()).index(v)
        label = interaction_type_to_label[edge_data['interaction_type']]
        edge_labels[u_index, v_index] = label

    data.edge_label = edge_labels  # Directly modify the data object

In [5]:
import torch.optim as optim

# Process validation data
val_MDG = build_multidigraph_from_csv('cleaned_relations_val_final.csv')

# Create a global node to index mapping
all_nodes = set(MDG.nodes()).union(set(val_MDG.nodes()))
global_node_to_index = {node: idx for idx, node in enumerate(all_nodes)}

# Function to map nodes in a graph to global indices
def map_nodes_to_global_indices(G, global_node_to_index):
    return [global_node_to_index[node] for node in G.nodes()]

val_data = from_networkx(val_MDG)

train_data.node_index = torch.tensor(map_nodes_to_global_indices(MDG, global_node_to_index), dtype=torch.long)
val_data.node_index = torch.tensor(map_nodes_to_global_indices(val_MDG, global_node_to_index), dtype=torch.long)
setup_edge_labels_with_no_interaction(val_MDG, interaction_type_to_label, val_data)
setup_edge_labels_with_no_interaction(MDG, interaction_type_to_label, train_data)

# Assuming the GCN class is defined
model = GCN(num_nodes, embedding_dim, num_classes)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

In [6]:
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.loader import DataLoader
from tqdm import tqdm

# Create a dataset of edges and their labels
edges = train_data.edge_index.t().tolist()  # List of [node_u, node_v]
labels = [train_data.edge_label[edge[0], edge[1]].item() for edge in edges]

edge_dataset = list(zip(edges, labels))
edge_loader = DataLoader(edge_dataset, batch_size=1024, shuffle=True)  # Adjust batch_size as needed

# Assuming val_data is structured similarly to train_data
val_edges = val_data.edge_index.t().tolist()
val_labels = [val_data.edge_label[edge[0], edge[1]].item() for edge in val_edges]

val_edge_dataset = list(zip(val_edges, val_labels))
val_edge_loader = DataLoader(val_edge_dataset, batch_size=1024, shuffle=False)  # You can adjust the batch size

from sklearn.metrics import f1_score, classification_report

inverse_interaction_type_to_label = {v: k for k, v in interaction_type_to_label.items()}

def validate():
    model.eval()
    total_loss = 0
    total_cross_entropy_loss = 0  # Initialize cross-entropy loss
    all_predictions = []
    all_true_labels = []

    with torch.no_grad():
        for batch in tqdm(val_edge_loader, desc='Validating'):
            edge_tensors, label_batch = batch
            node_u_list, node_v_list = edge_tensors[0].to(device), edge_tensors[1].to(device)
            label_batch = label_batch.to(device)

            output = model(val_data.to(device))
            edge_predictions = output[node_u_list].argmax(dim=1)  # Predicted classes

            # Calculate and accumulate cross-entropy loss
            cross_entropy_loss = criterion(output[node_u_list], label_batch)
            total_cross_entropy_loss += cross_entropy_loss.item()

            # Store predictions and true labels
            preds = edge_predictions.cpu().numpy()
            true_labels = label_batch.cpu().numpy()
            all_predictions.extend(preds)
            all_true_labels.extend(true_labels)

    # Calculate average cross-entropy loss over all batches
    avg_cross_entropy_loss = total_cross_entropy_loss / len(val_edge_loader)

    # Calculate F1 Score and classification report
    weighted_f1 = f1_score(all_true_labels, all_predictions, average='weighted')
    class_report = classification_report(all_true_labels, all_predictions, target_names=[inverse_interaction_type_to_label[i] for i in range(num_classes)], zero_division=0)

    print(f"Weighted F1 Score: {weighted_f1}\nClassification Report:\n{class_report}")
    print(f"Validation Cross-Entropy Loss: {avg_cross_entropy_loss}")

    return weighted_f1


In [7]:
# Assuming MDG is your training MultiDiGraph and val_MDG is your validation MultiDiGraph

# Step 1: Calculate Total Number of Unique Nodes
all_nodes = set(MDG.nodes()).union(set(val_MDG.nodes()))
num_nodes = len(all_nodes)

# Step 2: Identify Nodes Present Only in the Validation Set
train_nodes = set(MDG.nodes())
val_nodes = set(val_MDG.nodes())
unique_to_val = val_nodes.difference(train_nodes)

# Print or store the unique nodes to validation set
print("Nodes unique to validation set:", unique_to_val)


Nodes unique to validation set: set()


In [8]:
# Instantiate the model
model = GCN(num_nodes, embedding_dim, num_classes)
optimizer = optim.Adam(model.parameters(), lr=0.00001)  # Learning rate
criterion = torch.nn.CrossEntropyLoss()  # Loss function
# Check if CUDA (GPU support) is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Move your model to the chosen device
model = model.to(device)

def train():
    model.train()
    total_loss = 0

    for batch in tqdm(edge_loader, desc='Training'):
        edge_tensors, label_batch = batch
        
        node_u_list, node_v_list = edge_tensors[0].to(device), edge_tensors[1].to(device)
        label_batch = label_batch.to(device)

        optimizer.zero_grad()
        output = model(train_data.to(device))

        edge_predictions = output[node_u_list]  # Corrected indexing
        
        loss = criterion(edge_predictions, label_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(edge_loader)

Using device: cuda


In [9]:
# Early stopping parameters
patience = 10  # Number of epochs to wait for improvement before stopping
best_val_loss = float('inf')
epochs_no_improve = 0

# Set up the learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=5)

best_f1_score = 0
epochs_no_improve = 0
best_model_state = None

for epoch in range(1000):
    train_loss = train()  # Your training function
    val_f1_score = validate()  # Your validation function returning F1 score

    print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val F1 Score: {val_f1_score:.4f}')

    # Step the scheduler with the F1 score
    scheduler.step(val_f1_score)

    # Check for improvement
    if val_f1_score > best_f1_score:
        best_f1_score = val_f1_score
        epochs_no_improve = 0
        best_model_state = model.state_dict()  # Update the best model state
    else:
        epochs_no_improve += 1

    if epochs_no_improve >= patience:
        print("Early stopping triggered")
        break

# Save the best model state (outside of the loop)
if best_model_state is not None:
    torch.save(best_model_state, 'new_GCN_32_32_best.pth')
    print("Best model saved.")
else:
    print("No model improvement was observed.")

Validating: 100%|██████████| 18/18 [00:02<00:00,  7.85it/s]


Weighted F1 Score: 0.21846581191890446
Classification Report:
                     precision    recall  f1-score   support

    phosphorylation       0.04      0.16      0.06      1024
         expression       0.03      0.18      0.05       722
         inhibition       0.00      0.00      0.00      1745
binding/association       0.00      0.00      0.00       458
         activation       0.44      0.22      0.29      7342
           compound       0.02      0.01      0.01       694
        no_relation       0.33      0.25      0.28      6143

           accuracy                           0.19     18128
          macro avg       0.12      0.12      0.10     18128
       weighted avg       0.29      0.19      0.22     18128

Validation Cross-Entropy Loss: 2.0051238934199014
Epoch 1, Train Loss: 2.1183, Val F1 Score: 0.2185


Validating: 100%|██████████| 18/18 [00:02<00:00,  7.89it/s]


Weighted F1 Score: 0.2272651639176088
Classification Report:
                     precision    recall  f1-score   support

    phosphorylation       0.04      0.15      0.06      1024
         expression       0.02      0.16      0.04       722
         inhibition       0.00      0.00      0.00      1745
binding/association       0.01      0.00      0.00       458
         activation       0.45      0.23      0.30      7342
           compound       0.02      0.01      0.01       694
        no_relation       0.33      0.26      0.29      6143

           accuracy                           0.20     18128
          macro avg       0.12      0.12      0.10     18128
       weighted avg       0.29      0.20      0.23     18128

Validation Cross-Entropy Loss: 1.9844081070688036
Epoch 2, Train Loss: 2.0961, Val F1 Score: 0.2273


Validating: 100%|██████████| 18/18 [00:02<00:00,  7.20it/s]


...

Weighted F1 Score: 0.309663734099299
Classification Report:
                     precision    recall  f1-score   support

    phosphorylation       0.01      0.00      0.00      1024
         expression       0.00      0.00      0.00       722
         inhibition       0.18      0.01      0.01      1745
binding/association       0.00      0.00      0.00       458
         activation       0.45      0.39      0.42      7342
           compound       0.01      0.02      0.01       694
        no_relation       0.33      0.54      0.41      6143

           accuracy                           0.34     18128
          macro avg       0.14      0.14      0.12     18128
       weighted avg       0.31      0.34      0.31     18128

Validation Cross-Entropy Loss: 1.7395326561397977
Epoch 76, Train Loss: 1.7491, Val F1 Score: 0.3097


Validating: 100%|██████████| 18/18 [00:02<00:00,  8.22it/s]


Weighted F1 Score: 0.309663734099299
Classification Report:
                     precision    recall  f1-score   support

    phosphorylation       0.01      0.00      0.00      1024
         expression       0.00      0.00      0.00       722
         inhibition       0.18      0.01      0.01      1745
binding/association       0.00      0.00      0.00       458
         activation       0.45      0.39      0.42      7342
           compound       0.01      0.02      0.01       694
        no_relation       0.33      0.54      0.41      6143

           accuracy                           0.34     18128
          macro avg       0.14      0.14      0.12     18128
       weighted avg       0.31      0.34      0.31     18128

Validation Cross-Entropy Loss: 1.739369683795505
Epoch 77, Train Loss: 1.7487, Val F1 Score: 0.3097


Validating: 100%|██████████| 18/18 [00:02<00:00,  8.33it/s]

Weighted F1 Score: 0.309663734099299
Classification Report:
                     precision    recall  f1-score   support

    phosphorylation       0.01      0.00      0.00      1024
         expression       0.00      0.00      0.00       722
         inhibition       0.18      0.01      0.01      1745
binding/association       0.00      0.00      0.00       458
         activation       0.45      0.39      0.42      7342
           compound       0.01      0.02      0.01       694
        no_relation       0.33      0.54      0.41      6143

           accuracy                           0.34     18128
          macro avg       0.14      0.14      0.12     18128
       weighted avg       0.31      0.34      0.31     18128

Validation Cross-Entropy Loss: 1.739157756169637
Epoch 78, Train Loss: 1.7495, Val F1 Score: 0.3097
Early stopping triggered
Best model saved.


In [10]:
import pandas as pd
import torch
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx

# Load the best model
model_path = 'model_trial_0.pth'
model.load_state_dict(torch.load(model_path))
model = model.to(device)
model.eval()

# Load the validation dataset
val_MDG = build_multidigraph_from_csv('relations_val_final.csv')
val_data = from_networkx(val_MDG)
val_data.node_index = torch.arange(val_data.num_nodes)
setup_edge_labels_with_no_interaction(val_MDG, interaction_type_to_label, val_data)

# DataFrame to store the results
val_df = pd.read_csv('relations_val_final.csv')
predictions = []

# Make predictions
with torch.no_grad():
    for u, v in val_df[['starter_ID', 'receiver_ID']].itertuples(index=False):
        u_index = list(val_MDG.nodes()).index(u)
        v_index = list(val_MDG.nodes()).index(v)
        output = model(val_data.to(device))
        pred = output[u_index].argmax(dim=0).item()
        predictions.append(pred)

# Add predictions to the DataFrame
val_df['prediction'] = predictions

# Save the DataFrame with predictions
val_df.to_csv('validate_result_0.csv', index=False)


FileNotFoundError: [Errno 2] No such file or directory: 'model_trial_0.pth'